In [45]:
import earthaccess
import xarray as xr
import numpy as np
from PIL import Image, ImageEnhance

tspan = ("2024-09-22", "2024-09-28")
bbox = (-125., 32., -116., 38.)
clouds = (0, 50)

In [2]:
def enhancel3(rgb, scale = .01, vmin = 0.01, vmax = 1.02, gamma=.95, contrast=1.5, brightness=1.02, sharpness=2, saturation=1.1):
   
    rgb = rgb.where(rgb > 0)
    rgb = np.log(rgb / scale) / np.log(1 / scale)
    rgb = (rgb -  rgb.min()) / (rgb.max() - rgb.min())
    rgb = rgb * gamma
    img = rgb * 255
    img = img.where(img.notnull(), 0).astype("uint8")
    img = Image.fromarray(img.data)
    enhancer = ImageEnhance.Contrast(img)
    img = enhancer.enhance(contrast)
    enhancer = ImageEnhance.Brightness(img)
    img = enhancer.enhance(brightness)
    enhancer = ImageEnhance.Sharpness(img)
    img = enhancer.enhance(sharpness)
    enhancer = ImageEnhance.Color(img)
    img = enhancer.enhance(saturation)
    rgb[:] = np.array(img) / 255    
    return rgb

In [3]:
results = earthaccess.search_data(
    short_name="PACE_OCI_L2_SFREFL",
    temporal=tspan,
    bounding_box=bbox,
    # cloud_cover=clouds,
)
paths = earthaccess.open(results)
datatree = xr.open_datatree(paths[0])
dataset = xr.merge(datatree.to_dict().values())

QUEUEING TASKS | :   0%|          | 0/2 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/2 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/2 [00:00<?, ?it/s]

In [23]:
dataset = dataset.set_coords(("longitude", "latitude"))

In [28]:
dataset

<xarray.Dataset> Size: 1GB
Dimensions:        (number_of_bands: 286, number_of_reflective_bands: 286,
                    wavelength_3d: 122, number_of_lines: 1710,
                    pixels_per_line: 1272)
Coordinates:
  * wavelength_3d  (wavelength_3d) float64 976B 346.0 351.0 ... 2.258e+03
    longitude      (number_of_lines, pixels_per_line) float32 9MB ...
    latitude       (number_of_lines, pixels_per_line) float32 9MB ...
Dimensions without coordinates: number_of_bands, number_of_reflective_bands,
                                number_of_lines, pixels_per_line
Data variables: (12/25)
    wavelength     (number_of_bands) float64 2kB 315.0 316.0 ... 2.258e+03
    vcal_gain      (number_of_reflective_bands) float32 1kB ...
    vcal_offset    (number_of_reflective_bands) float32 1kB ...
    F0             (number_of_reflective_bands) float32 1kB ...
    aw             (number_of_reflective_bands) float32 1kB ...
    bbw            (number_of_reflective_bands) float32 1kB ...
    ...             ...
    clat           (number_of_lines) float32 7kB ...
    elat           (number_of_lines) float32 7kB ...
    csol_z         (number_of_lines) float32 7kB ...
    rhos           (number_of_lines, pixels_per_line, wavelength_3d) float32 1GB ...
    l2_flags       (number_of_lines, pixels_per_line) int32 9MB ...
    tilt           (number_of_lines) float32 7kB ...
Attributes: (12/45)
    title:                             OCI Level-2 Data SFREFL
    product_name:                      PACE_OCI.20240922T194620.L2.SFREFL.V3_...
    processing_version:                3.0
    history:                           l2gen par=/data12/sdpsoper/vdc/vpu31/w...
    instrument:                        OCI
    platform:                          PACE
    ...                                ...
    geospatial_lon_max:                -92.49064
    geospatial_lon_min:                -125.59848
    startDirection:                    Ascending
    endDirection:                      Ascending
    day_night_flag:                    Day
    earth_sun_distance_correction:     0.9930093288421631

In [24]:
rhos_rgb = dataset["rhos"].sel({"wavelength_3d": [645, 555, 371]})
rhos_rgb["channel"] = ("wavelength_3d", ["Reds", "Greens", "Blues"])
rhos_rgb = rhos_rgb.swap_dims({"wavelength_3d": "channel"})
rhos_rgb_enhanced = enhancel3(rhos_rgb)

In [25]:
rhos_rgb_max = rhos_rgb.max()
rhos_rgb_min = rhos_rgb.min()
rhos_rgb_enhanced = (rhos_rgb - rhos_rgb_min) / (rhos_rgb_max - rhos_rgb_min)

In [27]:
rhos_rgb_enhanced.plot.imshow(x='longitude', y='latitude')

ValueError: 'longitude' is not a dimension